In [216]:
import pandas as pd
import os
import sys
import pickle
import datetime
import time
import re
from sqlalchemy import create_engine
from sqlalchemy import text
import sqlalchemy
import psycopg2

In [217]:
STOP_WORDS = { "the",
    "and", "or", "the", "a", "an", "to", "of", "in", "on", "for", "with",
    "by", "at", "from", "as", "is", "are", "this", "that", 'tv', 'collection',
    'episodes' ,'episode', 'и', 'ну', 'не', 'ссср','shorts', 'y', 'kidsmania',
    'un', 'el', 'de', 'be', 'episodio'
}

In [218]:
#
def tokenize_clean(text):
    tokens = re.findall(r"\b[^\W_]+\b", text.lower(), flags=re.UNICODE)
    return [ t.strip() for t in tokens if not t.isdigit() and t not in STOP_WORDS ]

In [219]:
def get_intersect_ch(in_sentence, unigrams_s):
    #print('in_sentence', in_sentence)
    in_set = set(in_sentence)
    return(len(list(unigrams_s.intersection(in_set))))

In [220]:
def get_intersect(in_sentence, unigrams_s):
    in_set = set(in_sentence)
    return(list(unigrams_s.intersection(in_set)))

In [221]:
# Получаем исходные данные

In [222]:
conn_string = os.environ['PG_DB2']
db = create_engine(conn_string.replace('multdb', 'youtube'))
conn = db.connect()
req1 = '''select iyr.yt_reel_id, reel_name, yt_ch_name, cartoon, brand_id, product_id,language from intl_yt_reels iyr 
join intl_yt_channels2reels iyrcr 
on iyr.yt_reel_id = iyrcr.yt_reel_id 
join intl_yt_channels iyc 
on iyrcr.yt_ch_id = iyc.yt_ch_id 
where iyc."language" in ('США','Английский', 'Испанский', 'Арабский', 'Португальский','Китайский')
and iyr.removed = 0  '''
df_sql = pd.read_sql(text(req1), conn)
conn.close()

# Get uni and bigrams

In [223]:
# check "our" and "foreign" reels

In [224]:
print(len(df_sql[df_sql.product_id != '']))
print(len(df_sql[df_sql.product_id == '']))

3980
58829


In [225]:
# В качестве исходника берем не только название рила, но и название канала на которым он выложен:

In [226]:
df_sql['name'] = df_sql.yt_ch_name + ' '+df_sql.reel_name

In [227]:
df_sql["tokens"] = df_sql["name"].apply(tokenize_clean)

In [228]:
# Формируем список tokens для обоих вариантов

In [229]:
plus_scs = df_sql[df_sql.product_id != ''].tokens.to_list()
minus_scs = df_sql[df_sql.product_id == ''].tokens.to_list()

In [230]:
# Получаем список слов 

In [231]:
plus_words = [word.strip('.,:-').lower() for line in plus_scs for word in line]
minus_words = [word.strip('.,:-').lower()for line in minus_scs for word in line]

In [232]:
words_differ = set(plus_words) - set(minus_words)
len(words_differ)

769

In [233]:
# Итак, вот столько слов отличает оба списка и являются базой для разметки рилов

In [234]:
# Посмотрим частоту использования каждого слова для разметки:

In [235]:
result = pd.DataFrame({'word': plus_words})

result_df = (result['word'].value_counts().rename_axis('keyword').reset_index(name='count'))

result_df = result_df[result_df['keyword'].isin(words_differ)]
display(result_df.head(5))

,keyword,count
1,rockoons,1672
3,musipaches,1207
11,naranjas,830
17,مشمش,474
27,moo,280


In [236]:
print("All words:", len(result_df))
print("Single reel words:", len(result_df[result_df['count']==1]))
display(result_df.head(5))
display(result_df.tail(5))

All words: 769
Single reel words: 479


,keyword,count
1,rockoons,1672
3,musipaches,1207
11,naranjas,830
17,مشمش,474
27,moo,280


,keyword,count
2280,rocksongsforkids,1
2282,wheater,1
2283,menudo,1
2285,rara,1
2291,summers,1


In [237]:
# Bigrams

In [238]:
df_sql["bigrams"] = df_sql["tokens"].apply(lambda x: list(zip(x, x[1:])))
plus_bigrams = df_sql[df_sql.product_id != '']["bigrams"].explode().dropna().tolist()
minus_bigrams = df_sql[df_sql.product_id == '']["bigrams"].explode().dropna().tolist()
bigrams_differ = set(plus_bigrams)-set(minus_bigrams)
print(len(bigrams_differ))

5260


In [239]:
result = pd.DataFrame({'bi_word': plus_bigrams})

bigrams_df = (result['bi_word'].value_counts().rename_axis('keyword').reset_index(name='count'))

bigrams_df = bigrams_df[bigrams_df['keyword'].isin(bigrams_differ)]
display(bigrams_df.head(5))

,keyword,count
1,"(las, vacas)",830
2,"(vacas, naranjas)",830
7,"(عائلة, مشمش)",389
8,"(musipaches, musipaches)",367
9,"(rockoons, rockoons)",360


In [240]:
print("All bigrams:", len(bigrams_df))
print("Single reel bigrams:", len(bigrams_df[bigrams_df['count']==1]))
display(bigrams_df.head(5))
display(bigrams_df.tail(5))

All bigrams: 5260
Single reel bigrams: 3648


,keyword,count
1,"(las, vacas)",830
2,"(vacas, naranjas)",830
7,"(عائلة, مشمش)",389
8,"(musipaches, musipaches)",367
9,"(rockoons, rockoons)",360


,keyword,count
6249,"(witches, adventures)",1
6251,"(tallest, claymotions)",1
6253,"(summer, midst)",1
6254,"(midst, winter)",1
6255,"(winter, oomka)",1


# Clean unigram list

In [241]:
# get reels that were marked by more than one unigram (word)

In [242]:
unigrams_final = result_df.copy()

In [243]:
unigrams_set = set(unigrams_final.keyword.to_list())

In [244]:
reels_plus =  df_sql[df_sql.product_id != ''].copy()

In [245]:
reels_plus['ch2_unigrams'] = reels_plus['tokens'].apply(get_intersect_ch, args=(unigrams_set,)) 

In [246]:
reels_multi = reels_plus[reels_plus.ch2_unigrams >  1].copy() # was >1

In [247]:
# Let's get these words for each reel (intersection with unigrams_set):

In [248]:
reels_multi['unigrams'] = reels_plus['tokens'].apply(get_intersect, args=(unigrams_set,)) 
reels_multi = reels_multi[['yt_reel_id', 'tokens', 'unigrams', 'ch2_unigrams']]

In [249]:
# and get unique list of unigrams, that used more then one time for markup

In [250]:
unigrams_multi = reels_multi['unigrams'].to_list()
unigrams_multi_set = set([ i for j in unigrams_multi  for i in j ] )
print(len(unigrams_multi_set))

722


In [251]:
# let's make dict kind of: word, list of reel_ids that this word marked

In [252]:
unigram2reels = {}
for unigram in unigrams_multi_set:
    df_temp = reels_multi[reels_multi['tokens'].apply(lambda x: isinstance(x, list) and unigram in x)]
    unigram2reels[unigram] = set(df_temp.yt_reel_id.to_list())

In [253]:
subset_keys = [ k for k, v in unigram2reels.items()
    if not any( v < other_v for other_k, other_v in unigram2reels.items() if other_k != k ) ]  # strict subset, exclude itself

In [254]:
print(len(subset_keys))

86


In [255]:
##############

In [256]:
unigrams_final_set = set(subset_keys)

In [257]:
df_unigram2reels = pd.DataFrame( [(k, list(v)) for k, v in unigram2reels.items()], columns=['unigram', 'id'])

df_unigram2reels.columns = ['unigramm', 'ids']
df_unigram2reels['ids_num'] = df_unigram2reels['ids'].apply(len)

In [258]:
to_remove = []
for i in range(len(unigrams_final_set)):
    bi_1 = list(unigrams_final_set)[i]
    for j in range(i+1,len(unigrams_final_set)):
        bi_2 =  list(unigrams_final_set)[j]
        a = set(df_unigram2reels[df_unigram2reels.unigramm == bi_1].ids.to_list()[0])
        b = set(df_unigram2reels[df_unigram2reels.unigramm == bi_2].ids.to_list()[0])
        if(a == b):
            print(bi_1, bi_2)
            to_remove.append(bi_2)

deadbone sitcom
gamayun prophet
سأمسك الأذى
даша людоед
caramba apr23
caramba graciosas
drago trihead
apr23 graciosas
valle miel
casino caper
goddesses erinyes
المتحرك الصلصال
deeds journal
tzar firebird
彩泥行动 我们在学习数数
kidnapping hoglets


In [259]:
unigrams_final_set = (unigrams_final_set - set(to_remove))

In [260]:
##### Remove unigrams that detects single yt_reel_id

In [261]:
single_set = set(df_unigram2reels[df_unigram2reels['ids_num'] < 6].unigramm.to_list()) # really ???? was == 1 

print(len(single_set))

606


In [262]:
single_set = set(df_unigram2reels[df_unigram2reels['ids_num'] < 6].unigramm.to_list()) # really ???? was == 1 
print(len(single_set))
print(len(unigrams_final_set))
unigrams_final_set = unigrams_final_set - unigrams_final_set.intersection(single_set)
print(len(unigrams_final_set))

606
71
39


In [263]:
#unigrams_final_set

# Clean bigrams from those that contains unigrams from set

In [264]:
bigrams_df['words_exst'] = bigrams_df.keyword.apply(lambda x: x[0] in unigrams_final_set or x[1] in unigrams_final_set)

bigrams_df[(bigrams_df['words_exst'] == False) & (bigrams_df['count'] > 3) ].head(3)

,keyword,count,words_exst
1,"(las, vacas)",830,False
29,"(infantiles, canciones)",192,False
31,"(عائلة, أبريكوت)",162,False


In [265]:
bigrams_df[(bigrams_df['words_exst'] == True)].sample(3)

,keyword,count,words_exst
3774,"(مشمش, لغز)",1,True
1402,"(rockoons, theatre)",2,True
3030,"(marionetas, lápiz)",1,True


In [266]:
# Разметили биграммы - теперь есть отметка тех, где используются униграммы - те которые можно убрать

#unigrams_final = unigrams_df.copy()
bigrams_final = bigrams_df.copy()

In [267]:
len(bigrams_final[bigrams_final.words_exst == False]) # words_exst == True - don't use

3936

In [268]:
bigrams_final

,keyword,count,words_exst
1,"(las, vacas)",830,False
2,"(vacas, naranjas)",830,True
7,"(عائلة, مشمش)",389,True
8,"(musipaches, musipaches)",367,True
9,"(rockoons, rockoons)",360,True
...,...,...,...
6249,"(witches, adventures)",1,False
6251,"(tallest, claymotions)",1,True
6253,"(summer, midst)",1,False
6254,"(midst, winter)",1,False


# Clean bigram list

In [271]:
bigrams_set = set(bigrams_final[bigrams_final.words_exst==False].keyword.to_list())

In [272]:
reels_plus['ch2_bigrams'] = reels_plus['bigrams'].apply(get_intersect_ch, args=(bigrams_set,)) 
reels_plus['sel_bigrams'] = reels_plus['bigrams'].apply(get_intersect, args=(bigrams_set,)) 

In [273]:
reels_multi = reels_plus[reels_plus.ch2_bigrams >  1].copy() # was >1

In [274]:
reels_multi = reels_multi[['yt_reel_id', 'bigrams', 'sel_bigrams', 'ch2_bigrams']]

In [275]:
bigrams_multi = reels_multi['sel_bigrams'].to_list()
bigrams_multi_set = set([ i for j in bigrams_multi  for i in j ] )
len(bigrams_multi_set)

3446

In [276]:
bigram2reels = {}
for bigram in bigrams_multi_set:
    df_temp = reels_multi[reels_multi['bigrams'].apply(lambda x: isinstance(x, list) and bigram in x)]
    bigram2reels[bigram] = set(df_temp.yt_reel_id.to_list())

In [277]:
#bigram2reels

In [278]:
subset_bi_keys = [ k for k, v in bigram2reels.items()
    if not any( v < other_v for other_k, other_v in bigram2reels.items() if other_k != k ) ]  # strict subset, exclude itself

In [279]:
print(len(subset_bi_keys))
bigrams_final_set = set(subset_bi_keys)

953


In [280]:
df_bigram2reels = pd.DataFrame( [(k, list(v)) for k, v in bigram2reels.items()], columns=['bigram', 'id'])

In [281]:

df_bigram2reels['ids_num'] = df_bigram2reels['id'].apply(len)
single_set_bi = set(df_bigram2reels[df_bigram2reels['ids_num'] < 1  ].bigram.to_list()) # was <2 !!!

print(len(single_set_bi))
print(len(bigrams_final_set))
bigrams_final_set = bigrams_final_set - bigrams_final_set.intersection(single_set_bi)
print(len(bigrams_final_set))

0
953
953


In [282]:
to_remove = []
for i in range(len(bigrams_final_set)):
    print(i, end =' - ')
    bi_1 = list(bigrams_final_set)[i]
    for j in range(i+1,len(bigrams_final_set)):
        bi_2 =  list(bigrams_final_set)[j]
        a = set(df_bigram2reels[df_bigram2reels.bigram == bi_1].id.to_list()[0])
        b = set(df_bigram2reels[df_bigram2reels.bigram == bi_2].id.to_list()[0])
        if(a == b):
            print(bi_1, bi_2)
            to_remove.append(bi_2)

('الإطلاق', 'special') ('أبريكوت', 'أحلى')
('الإطلاق', 'special') ('العيد', 'الكبرى')
('الإطلاق', 'special') ('الكبرى', 'لعائلة')
('الإطلاق', 'special') ('عيد', 'على')
('الإطلاق', 'special') ('special', 'eid')
('الإطلاق', 'special') ('حلقة', 'عيد')
('الإطلاق', 'special') ('أحلى', 'حلقة')
('الإطلاق', 'special') ('cow', 'مفاجأة')
('الإطلاق', 'special') ('لعائلة', 'أبريكوت')
('caracola', 'almohada') ('almohada', 'dibujos')
('trouble', 'nursery') ('solving', 'trouble')
('caracola', 'dibujos') ('plastilina', 'caracola')
('números', 'mágicos') ('mágicos', 'del')
('wind', 'instruments') ('kids', 'wind')
('aprenda', 'os') ('massinha', 'aprenda')
('aprenda', 'os') ('os', 'números')
('aprenda', 'os') ('números', 'massinha')
('aprenda', 'os') ('felizes', 'números')
('massinha', 'aprenda') ('os', 'números')
('massinha', 'aprenda') ('números', 'massinha')
('massinha', 'aprenda') ('felizes', 'números')
('pinzas', 'plastilina') ('plastilina', 'dibujos')
('planes', 'learn') ('transport', 'traffic')
('

In [284]:
bigrams_final_set = bigrams_final_set - set(to_remove)
print(len(bigrams_final_set))

529


In [513]:
#bigrams_final_set
#unigrams_final_set

In [514]:
with open("bigrams_final_set.pkl", 'wb') as file:
    # Serialize and write the set to the file
    pickle.dump(bigrams_final_set, file)
file.close()

In [515]:
with open("unigrams_final_set.pkl", 'wb') as file:
    # Serialize and write the set to the file
    pickle.dump(unigrams_final_set, file)
file.close()

# Combine uni- and bi-grams

In [285]:
reels_plus =  df_sql[df_sql.product_id != ''].copy()

In [286]:
reels_plus['ch2_bigrams'] = reels_plus['bigrams'].apply(get_intersect_ch, args=(bigrams_set,)) 
reels_plus['sel_bigrams'] = reels_plus['bigrams'].apply(get_intersect, args=(bigrams_set,)) 
reels_multi = reels_plus[reels_plus.ch2_bigrams >  1].copy() # was >1
reels_multi = reels_multi[['yt_reel_id', 'bigrams', 'sel_bigrams', 'ch2_bigrams']]

In [287]:
bigram2reels = {}
for bigram in bigrams_final_set:
    df_temp = reels_multi[reels_multi['bigrams'].apply(lambda x: isinstance(x, list) and bigram in x)]
    bigram2reels[bigram] = set(df_temp.yt_reel_id.to_list())

In [288]:
df_bigram2reels = pd.DataFrame( [(k, list(v)) for k, v in bigram2reels.items()], columns=['bigram', 'id'])

In [289]:

reels_plus['ch2_unigrams'] = reels_plus['tokens'].apply(get_intersect_ch, args=(unigrams_set,)) 
reels_multi = reels_plus[reels_plus.ch2_unigrams >  1].copy() # was >1
reels_multi['unigrams'] = reels_plus['tokens'].apply(get_intersect, args=(unigrams_set,)) 
reels_multi = reels_multi[['yt_reel_id', 'tokens', 'unigrams', 'ch2_unigrams']]

In [290]:
unigram2reels = {}
for unigram in unigrams_final_set:
    df_temp = reels_multi[reels_multi['tokens'].apply(lambda x: isinstance(x, list) and unigram in x)]
    unigram2reels[unigram] = set(df_temp.yt_reel_id.to_list())

In [291]:
df_unigram2reels = pd.DataFrame( [(k, list(v)) for k, v in unigram2reels.items()], columns=['unigram', 'id'])
df_unigram2reels.columns = ['unigramm', 'ids']


In [292]:
df_unigram2reels.columns = ['x_grams', 'ids']
df_bigram2reels.columns = ['x_grams', 'ids']

In [293]:
df_grams2reels = pd.concat([df_unigram2reels,df_bigram2reels])

In [294]:
grams_set = set(df_grams2reels['x_grams'].to_list())

In [295]:
to_remove = []
for i in range(len(grams_set)):
    print(i, end = ' - ')
    bi_1 = list(grams_set)[i]
    for j in range(i+1,len(grams_set)):
        bi_2 =  list(grams_set)[j]
        a = set(df_grams2reels[df_grams2reels.x_grams == bi_1].ids.to_list()[0])
        b = set(df_grams2reels[df_grams2reels.x_grams == bi_2].ids.to_list()[0])
        if(a >= b):
            print(bi_1, bi_2)
            to_remove.append(bi_2)

الخمول ('نوم', 'بهدوء')
deeds ('journal', 'orange')
plastideas ('jugando', 'transformándote')
plastideas ('las', 'plantas')
plastideas ('qué', 'animal')
plastideas ('guauguau', 'aprende')
plastideas ('mágica', 'con')
plastideas ('acción', 'para')
plastideas ('instrumentos', 'sorprendentes')
plastideas ('vehículos', 'ayudantes')
plastideas ('musicales', 'estreno')
الراكون ('كرتون', 'غنائي')
الراكون ('القراصنة', 'كرتون')
الراكون ('ندف', 'الثلج')
الراكون ('الفوانيس', 'كرتون')
الراكون ('الحقيبة', 'كرتون')
الراكون ('المكنسة', 'الكهربائية')
الراكون ('كيدو', 'الحان')
الراكون ('الريشة', 'الطائرة')
الراكون ('المظلة', 'كرتون')
الراكون ('الخشخيشة', 'كرتون')
الراكون ('التاج', 'كرتون')
الراكون ('للأطفال', 'الحان')
الراكون ('الفرشاة', 'كرتون')
الراكون ('عصا', 'الطبل')
الراكون ('الازرار', 'كرتون')
musipaches ('idéntico', 'diferente')
musipaches ('naturales', 'dibujos')
musipaches ('columpio', 'dibujos')
musipaches ('popote', 'peine')
musipaches ('redondos', 'dibujos')
musipaches ('ligas', 'dibujos')


In [296]:
print(len(grams_set))
print(len(set(to_remove)))

568
128


In [297]:
final_grams_set = (grams_set - set(to_remove))
print(len(final_grams_set))

440


In [298]:
with open("final_grams_set.pkl", 'wb') as file:
    # Serialize and write the set to the file
    pickle.dump(final_grams_set, file)
file.close()

In [299]:
final_grams_set

{('4super', 'toonstv'),
 ('about', 'family'),
 ('above', 'below'),
 ('action', 'packed'),
 ('activities', 'new'),
 ('adventure', 'educational'),
 ('adventure', 'music'),
 ('al', 'desafío'),
 ('almuerzo', 'saludable'),
 ('alone', 'together'),
 ('along', 'across'),
 ('always', 'never'),
 ('amigos', 'dasha'),
 ('amigos', 'do'),
 ('amigos', 'grey'),
 ('animada', 'educativa'),
 ('animados', 'cancioneseducativas'),
 ('animados', 'niños'),
 ('animal', 'learn'),
 ('animales', 'cuántas'),
 ('animales', 'guauguau'),
 ('animales', 'quién'),
 ('animals', 'cartoon'),
 ('animals', 'how'),
 ('animals', 'kittens'),
 ('animals', 'learn'),
 ('animals', 'teeth'),
 ('animals', 'who'),
 ('animals', 'whos'),
 ('animals', 'woofwoof'),
 ('antifaz', 'dibujos'),
 ('aprenda', 'e'),
 ('aprenda', 'os'),
 ('apresurar', 'dibujos'),
 ('arctic', 'adventure'),
 ('arriba', 'abajo'),
 ('arte', 'dibujos'),
 ('atrapar', 'dibujos'),
 ('aventuras', 'casitas'),
 ('ayudando', 'amigos'),
 ('babies', 'new'),
 ('baby', 'ti'),
 ('

# Check detection by UNI- and BI-grams

In [300]:
reels_plus = df_sql[df_sql.product_id != '']
reels_minus = df_sql[df_sql.product_id == '']

In [301]:
reels_plus = reels_plus.assign(detected = 0)

In [302]:
for gram in final_grams_set:
    if type(gram) == str:
        print('str:', gram)
        reels_plus.loc[:,'detected'] += reels_plus['tokens'].apply(lambda cell: int(gram in cell))
    else:
        print('touple:', gram)
        reels_plus.loc[:,'detected'] += reels_plus['bigrams'].apply(lambda cell: int(gram in cell))

touple: ('الإطلاق', 'special')
touple: ('caracola', 'almohada')
touple: ('trouble', 'nursery')
touple: ('massinha', 'au')
str: récord
touple: ('caracola', 'dibujos')
touple: ('números', 'mágicos')
touple: ('wind', 'instruments')
touple: ('aprenda', 'os')
touple: ('favorite', 'games')
touple: ('pinzas', 'plastilina')
touple: ('planes', 'learn')
touple: ('والأخيرة', 'صخرة')
touple: ('new', 'record')
touple: ('أغاني', 'أطفال')
touple: ('whos', 'fastest')
touple: ('who', 'biggest')
touple: ('games', 'boys')
touple: ('games', 'theatre')
touple: ('juegos', 'deportivos')
touple: ('лентяйка', 'василиса')
touple: ('paint', 'brushes')
touple: ('siete', 'mapaches')
touple: ('colección', 'grande')
touple: ('roccoons', 'musical')
touple: ('الحلقة', 'طريقة')
touple: ('escondite', 'busca')
touple: ('al', 'desafío')
touple: ('pirates', 'abc')
touple: ('concurso', 'talentos')
touple: ('عائلة', 'المشمش')
touple: ('peine', 'ruedas')
touple: ('walk', 'stand')
touple: ('juguetes', 'bonitos')
touple: ('timi

In [303]:
reels_plus[reels_plus.detected <1]

,yt_reel_id,reel_name,yt_ch_name,cartoon,brand_id,product_id,language,name,tokens,bigrams,detected
